In [1]:
import os
import json
from bs4 import BeautifulSoup

In [8]:
def getHtmlFileNames():
    html_path = "year_htmls/"
    html_file_names = os.listdir(html_path)
    html_file_names = [_ for _ in sorted(html_file_names) if _.endswith(".html")]
    #
    return html_file_names

In [9]:
def openHtml(file_name):
    html_path = "year_htmls/{}".format(file_name)
    with open(html_path, "r", encoding='UTF-8') as f:
        html = f.read()
    #
    return html

In [10]:
def soupHtml(html):
    #
    soup = BeautifulSoup(html, "lxml")
    #
    return soup

In [11]:
def splitTrs(soup):
    # 각 순위별 tag들을 split하여 돌려준다.
    trs = soup.find_all("tr")
    # 첫 줄은 header이니 삭제.
    trs = trs[1:]
    #
    return trs

In [12]:
def parseAtr(a_tr):
    # ranking
    try:
        ranking = a_tr.find("span", class_="rank top").get_text(strip=True)
    except:
        ranking = a_tr.find("span", class_="rank").get_text(strip=True)
        
    # title_num
    title_num = a_tr.find("button", class_="btn_icon like")["data-song-no"]
    
    # title_name
    title_name = a_tr.find("div", class_="ellipsis rank01").strong.get_text(strip=True)
    
    # album_name 
    album_name = a_tr.find("div", class_="ellipsis rank03").find("a", class_="fc_mgray").get_text(strip=True)

    # album num
    album_num = a_tr.find("div", class_="ellipsis rank03").find("a")["href"]
    album_num = album_num.replace("javascript:melon.link.goAlbumDetail('", "").replace("');", "")

    # artist
    artist_obj = a_tr.find("div", class_="ellipsis rank02").find_all("a")
    artist = []
    for tag in artist_obj:
        artist_name = tag.get_text(strip=True)
        artist_num = tag["href"].replace("javascript:melon.link.goArtistDetail('", "").replace("');", "")
        if (artist_num, artist_name) not in artist:
            artist.append((artist_num, artist_name))
    
    # last line
    line = {"ranking": ranking, "title_num": title_num, "title_name": title_name,
           "album_num": album_num, "album_name": album_name, "artists": artist}
    
    return line

In [13]:
target_year = "2023"
html_file_names = getHtmlFileNames()
print("files: {}".format(len(html_file_names)))

files: 24


In [16]:
lines = []
for cnt, a_name in enumerate(html_file_names, 1):
    #yearweek = "{}{:02}".format(target_year, cnt)
    period = a_name.replace(".html", "")
    
    print(period)
    
    html = openHtml(a_name)
    soup = soupHtml(html)
    trs = splitTrs(soup)
    for a_tr in trs:
        line = parseAtr(a_tr)
        line["period"] = period
        lines.append(line)
        

2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023


In [19]:
print("total {} lines".format(len(lines)))

total 1898 lines


In [22]:
file_name = "parsed_year_chart_html/{}.jsonl".format(target_year)

In [23]:
with open(file_name, "w", encoding="utf-8") as of:
    for line in lines:
        print(json.dumps(line, ensure_ascii=False), file=of)